# PyLabRobot Turbidostat



##### Purpose:
This notebook is meant to be a comprehensive notebook for a turbidostat. 


##### Instructions:
Currently it is in the simulation configuration, in order to set it for a real experiment:
- change the backend
- change the deck to the apropriate robot configuration.
- modify the waste removal
- modify the media renewal
- determine SOP for water exchange (to prevent bleach accumulation in wash wells)
- Attach PLR plate reader to 'fake' plate reader class


##### Contents:
- SQLite database (via SQLalchemy)
- graphing feature for SQL data
- three different controllers for the turbidostats:
    - PID & k estimation control
    - k estimation control
    - direct proportional control
- plate & LiquidHandler setup
- PLR logic for turbidostat
    - dilute_96w_plate
    - mix
    - wash
- logic for cleaning tips for continous usage and no tip_switching

Required import beyond PLR/native python:
- seaborn
- sqlalchemy
- numpy
- pandas
- sqlite3



In [ ]:
# Run this code inn order to strip all outputs upon git add & commit
! pip install nbstripout
! nbstripout --install

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling import LiquidHandlerChatterboxBackend
from pylabrobot.liquid_handling.backends import LiquidHandlerBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck
from pylabrobot.resources.hamilton import HamiltonSTARDeck
from pylabrobot.resources import Deck, Coordinate

from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *
from pylabrobot.resources.agenbio.plates import AGenBio_4_troughplate_75000_Vb

from pylabrobot.resources.resource_stack import ResourceStack
from pylabrobot.resources.plate import Lid


from pylabrobot.resources import set_tip_tracking, set_volume_tracking, set_cross_contamination_tracking
set_tip_tracking(True), set_volume_tracking(True)

from pylabrobot.resources import (
    corning_96_wellplate_360ul_flat,
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    nest_12_reservoir_15ml,
    nest_96_wellplate_2ml_deep,
    corning_6_wellplate_16point8ml_flat,
    Cor_96_wellplate_360ul_Fb,
    Cor_96_wellplate_360ul_Fb_Lid,
)


# import opentrons
import seaborn
import time
import logging
import pandas as pd
import numpy as np
import json
import os


# For database construction & usage
from sqlalchemy import create_engine, Column, Integer, Float, String, ForeignKey, desc, func, asc
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
import sqlite3



In [ ]:
cptl_alphabet = [chr(i) for i in range(65, 91)]

In [ ]:
# Instantiate Database with sqlalchemy (more friendly for future users/swtiching the sytem in the future.)

Base = declarative_base()

class Plate(Base):
    __tablename__ = 'plates'
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    num_rows = Column(Integer, nullable=False)
    num_cols = Column(Integer, nullable=False)
    readings = relationship("Reading", back_populates="plate")
    wells = relationship("Well", back_populates="plate")


class Well(Base):
    __tablename__ = 'wells'
    id = Column(Integer, primary_key=True)
    row = Column(String, nullable=False)
    col = Column(Integer, nullable=False)
    readings = relationship("Reading", back_populates="well")
    plate_id = Column(Integer, ForeignKey('plates.id'), nullable=False)
    plate = relationship('Plate', back_populates='wells')


class Reading(Base):
    __tablename__ = 'readings'
    id = Column(Integer, primary_key=True)
    timestamp = Column(Float, default=lambda: time.time())  # UNIX timestamp
    plate_id = Column(Integer, ForeignKey('plates.id'))
    well_id = Column(Integer, ForeignKey('wells.id'))
    od = Column(Float)
    k_estimate = Column(Float)
    transfer_vol_frac = Column(Float)

    plate = relationship("Plate", back_populates="readings")
    well = relationship("Well", back_populates="readings")

# Create the database and session



# TO reset the database for testing to save data past each run.
# remove if data should be saved
# TODO: will change file handling when set into production code.

db_path = "turbidostat.db"

if os.path.exists(db_path):
    os.remove(db_path)


engine = create_engine("sqlite:///turbidostat.db")
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()



In [ ]:
def db_add_plate(session, plate):
    plate_db_obj = Plate(name=plate.name, num_rows=plate.num_items_y, num_cols=plate.num_items_x)
    session.add(plate_db_obj)
    session.commit()


def db_add_wells_from_plate(session, plate_name):
    # get a plate that is a database object, not a py lab robot object
    db_plate = session.query(Plate).filter_by(name=plate_name).first()     # there should only be 1, but this returns the actual object, not a list
    rows = db_plate.num_rows
    cols = db_plate.num_cols
    for row_index in range(rows):
        for col in range(cols):
            session.add(Well(plate=db_plate, row=cptl_alphabet[row_index], col=col+1))
            logging.info(f'added well to database: plate: {db_plate.name}. row, col: {cptl_alphabet[row_index]}, {col + 1}')
    session.commit()
    

def db_add_reading(session, reading, well_row, well_col, plate_name):
    '''
    ENSURE WELL EXISTS BEFORE ATTEMPTING TO ADD A READING.

    well_row should be an int, it is typecast to a capital letter in this function.
    '''
    od, k_est, transfer_vol_frac = reading

    db_well = session.query(Well).join(Well.plate).filter(
        Well.row == cptl_alphabet[well_row],
        Well.col == well_col,
        Plate.name == plate_name
        ).first()
    
    if db_well is None:
        logging.warning(f'Well does not exist. plate_name: {plate_name}. row, col: {cptl_alphabet[well_row]}, {well_col}')
        logging.warning(f'Adding well to database with same characteristics.')
        db_plate = session.query(Plate).filter_by(name=plate_name).first()
        db_well = Well(plate=db_plate, row=cptl_alphabet[well_row], col=well_col)
        session.add(db_well)

    session.add(Reading(well=db_well,                  # ORM Well object
        plate=db_well.plate,
        od=od,
        k_estimate=k_est,
        transfer_vol_frac=transfer_vol_frac
    ))
    logging.info(f'added reading to db: plate: {db_well.plate.name}, (row, col): ({db_well.row}, {db_well.col}), od: {od}, k_est: {k_est}, transfer_vol_frac: {transfer_vol_frac}')

    session.commit()


### Set up the Deck and Visualizer

In [ ]:
async def setup_lh() -> LiquidHandler:
    lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=OTDeck())

    await lh.setup()

    vis = Visualizer(resource=lh)
    await vis.setup()
    return lh

'num_rails', 'size_x', 'size_y', and 'size_z'

### Add labware to the deck

* 1 culture plates (corning 96 well plates)
* 1 tip racks
* 3 media plates (deep well plates)
* Plate position for reader tray
* Wash positions (single-channel waste, bleach, 4 waters)

You may need to create custom definitions for the media plates and wash positions (we can go over this)

The OT-2 only has 9 positions but we will have at least 12 for the ***** system


In [ ]:
async def deck_setup(lh: LiquidHandler, fill=True) -> list:
    # Declare a tip rack object - using opentrons for right now.
    tip_rack_0 =  opentrons_96_tiprack_300ul("tip_rack_0")
    tip_rack_1 =  opentrons_96_tiprack_300ul("tip_rack_1")

    # Assign the tip rack to spot 1
    lh.deck.assign_child_at_slot(tip_rack_0, 7)
    lh.deck.assign_child_at_slot(tip_rack_1, 8)

    tip_racks = [tip_rack_0, tip_rack_1]

    # Declare a plate object
    culture_plate_0 =  Cor_96_wellplate_360ul_Fb("culture_plate_0", with_lid=False)
    culture_plate_1 =  Cor_96_wellplate_360ul_Fb("culture_plate_1", with_lid=False)

    culture_plates = [
        culture_plate_0,
        culture_plate_1,
    ]


    # bleach plate /w water to rinse to ensure residual bacteria in tips are cleaned
    wash = AGenBio_4_troughplate_75000_Vb('wash')
    lh.deck.assign_child_at_slot(wash, 9)

    # instantiate media plates w/ lids & add to deck
    media_plates = [
        Cor_96_wellplate_360ul_Fb(f"media_plate_{i}", with_lid=False) for i in range(4)
    ]


    # for visualization purposes, ensure filling of tubes, set properly
    # ================================

    # fill all tubes w/ various fludids 
    if fill:
        for j in range(1, 13):
            for i in range(8):
                # Using the f-string allows us to iterate through 
                # wells A1 -> B1 -> C1 -> ... -> H1
                # all in one line of code!
                
                # set arbitrary volume for culture plates
                for media_plate in media_plates:
                    media_plate[f'{cptl_alphabet[i]}{j}'][0].tracker.set_liquids([("media", 300)])
                culture_plate_1[f'{cptl_alphabet[i]}{j}'][0].tracker.set_liquids([("culture_1", 150)])
                culture_plate_0[f'{cptl_alphabet[i]}{j}'][0].tracker.set_liquids([("culture_0", 150)])



        wash['A1'][0].tracker.set_liquids([('waste', 0)])
        wash['A2'][0].tracker.set_liquids([('bleach', 30000)])
        wash['A3'][0].tracker.set_liquids([('water', 30000)])
        wash['A4'][0].tracker.set_liquids([('waste', 30000)])


    # ================================

    return tip_racks, wash, media_plates, culture_plates


In [ ]:
# setup lid stacking for each of the media plates

def setup_stacks(media_plates, culture_plates, lh: LiquidHandler, media_slots=None, culture_slots=None):
    '''
    ALSO adds lids to each container as part of the stack not an attribute of the plates themselves
    '''

    if media_slots is None:
        media_slots = [i for i in range(1,5)]
    
    if culture_slots is None:
        culture_slots = [5, 6]

    media_stacks = []

    for i, media_plate in enumerate(media_plates):
        media_stacks.append(ResourceStack(name=f'media_stack_{i}', direction='z', resources=[media_plate]))
        lh.deck.assign_child_at_slot(media_stacks[i], media_slots[i])
    
    culture_stacks = []

    for i, culture_plate in enumerate(culture_plates):
        culture_stacks.append(ResourceStack(name=f'culture_stack_{i}', direction='z', resources=[culture_plate]))
        lh.deck.assign_child_at_slot(culture_stacks[i], culture_slots[i])
    
    return media_stacks, culture_stacks

In [ ]:
class PlateReader:
    def __init__(self, is_sim, output_file=None, session=None, data_sim=None):
        self.is_sim = is_sim
        self.output_file = output_file
        self.session = session
        self.data_sim = data_sim

        # self.PLR_pr = ... etc.
        # configure to attach to a PLR plate reader, save as an instance variable

    def aquire_data(self, plate_stack, mean_absorbance=0.5, std_dev=0.1, timedelta=0):
        '''
        If simulation, simulate data, else, call the plate reader and read the plate. returns the filename where the .csv result from the plate reader returns


        '''
        plate = plate_stack.children[0]
        # Get rows and columns from the plate
        if self.is_sim:
            self.data_sim.generate_data(bac_plate=plate, output_file=self.output_file)

        # not a simulation
        else:
            # gripper.move_labware(labware=plate, dest_spot=)
            # PLR Plate reader call
            # dont use the sim_data/ directory, just use the output file and re-write all of the data.
            # get the data and pass int .csv, return the filename
            pass

    def get_output_file(self):
        return self.output_file

    def get_index(self):
        return self.index

In [ ]:
class DataSimulator:
    def __init__(self, session=None, rand_nor_mean=1, rand_nor_std=0.05):
        self.session = session 
        self.rand_nor_mean = rand_nor_mean
        self.rand_nor_std = rand_nor_std


    def generate_data(self, bac_plate, output_file):
        ''' 
        Assuming that output_file already has a .csv file-extension on it, so that it does not need to  
        '''
        rows = bac_plate.num_items_y
        cols = bac_plate.num_items_x
        self.output_file = output_file

        data = []
        for row in range(rows):
            for col in range(cols):
                if self.session:
                    # get the last reading object for this cell
                    last_reading = self.session.query(Reading).join(Reading.well).join(Well.plate).filter(
                        Plate.name == bac_plate.name,
                        Well.col == col + 1,
                        Well.row == cptl_alphabet[row],
                    ).order_by(desc(Reading.timestamp)).first()
                    if last_reading:
                        last_od = last_reading.od

                        # dilution step, since OD ~= bact. conc., then M1V1 = M2V2
                        dilution_factor = last_reading.transfer_vol_frac
                        
                        last_od = ((last_od * 1)/(1 + dilution_factor))

                        # hard code value for simulation
                        randomization_factor = np.random.normal(
                            loc=self.rand_nor_mean, scale=self.rand_nor_std)

                        # if last_od <= 0:
                        #     logging.critical(f'od registed as negative (od: {last_od}), reverted to 0.1 info: plate: {plate.name}, well coords: {cptl_alphabet[row]}{col+1}')
                        #     last_od = 0.1

                        dt = 1200
                        k = last_reading.k_estimate
                        absorbance_value = last_od * np.exp((dt/3600)*k) * randomization_factor
                    else:
                        absorbance_value = max(np.random.normal(loc=self.rand_nor_mean, scale=self.rand_nor_std), 0.1)

                    # additional perturbation for sim testing
                    # if 30 <= self.index <= 60: 
                    #     absorbance_value = absorbance_value * 1.3
                data.append(absorbance_value)

        self.format_csv(measurements=data, output_file=output_file)

    def format_csv(self, measurements, output_file):
        test_name = 'Testname: test_001'
        date = "Date: 5/26/2025 Time: 4:38:44 PM (UTC--4)"
        id = 'ID1: sample_plate_0 ID2: - ID3: 0x33741382a'
        channels = 'No. of Channels / Multichromatics: 1'
        cycles = 'No. of Cycles: 1'
        end = 'End_of_header'

        chromatic = "Chromatic: 1"
        cycle = "Cycle: 1"


        output = [test_name, date, id, channels, cycles, end, "", chromatic, cycle]
        for i in range(8):
            print(cptl_alphabet[i])
            for j in range(1, 13):
                if len(str(j)) == 1:
                    well = cptl_alphabet[i] + '0' + str(j)
                else:
                    well = cptl_alphabet[i] + str(j)
                well = well + f': {measurements[i*12 + j - 1]}'
                output.append(well)

        df = pd.DataFrame(output)
        df.to_csv(output_file)
        


In [ ]:
class TurbController(object):
    id_counter = 0

    def __init__(self, setpoint=0.0, init_od=1e-6, output_limits=(0, 0.5), db_session=None):
        self.output_limits = output_limits       # TODO: adjust for valid values, such as the max of the 96 well plate, make it a variable
        self.setpoint = setpoint
        self.od = init_od
        self.state = {'update_time': time.time(), 'od':init_od}
        self.state_history = [self.state]
        self.name = str(self.id_counter)
        self.__class__.id_counter += 1
        self.ever_updated = False
        self.db_session = db_session

    def step(self, delta_time=None, od_meas=None, last_transfer_vol_frac=None):
        if delta_time is None:  # use real time
            self.state = {'update_time': time.time()}
        else:
            self.state = {'update_time': self._last_time() + delta_time}
        delta_time = self.state['update_time'] - self._last_time()
        transfer_vol_frac = self._step(
            delta_time, od_meas, last_transfer_vol_frac)
        # limit output
        min_out, max_out = self.output_limits
        transfer_vol_frac = min(max_out, max(min_out, transfer_vol_frac))
        self.state.update(
            {'od': self.od, 'delta_time': delta_time, 'output': transfer_vol_frac})
        self.state_history.append(self.state)
        self.ever_updated = True
        return transfer_vol_frac

    def _last_time(self):
        return self.state_history[-1]['update_time']
    
    def history(self):
        # omit initial state
        return self.state_history[1:] if self.state_history else []

    def last_known_od(self):
        last_state = self.state_history[-1]
        return last_state.get('od', self.od)

    def last_known_output(self):
        last_state = self.state_history[-1]
        return last_state.get('output', 0)

    def scrape_history(self, key, fill_value=None):
        return [state.get(key, fill_value) for state in self.history()]

    def __call__(self, *args, **kwargs):
        return self.step(*args, **kwargs)  # default to real time

    def save(self, save_dir='controller_history', filename=None):
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        if not os.path.isdir(save_dir):
            raise ValueError('Controller save directory is not a directory')
        if filename is None:
            filename = self.name + '.turbhistory'
        path = os.path.join(save_dir, filename)
        with open(path, 'w+') as f:
            f.write(json.dumps(self.state_history))

    def load(self, from_dir='controller_history', filename=None):
        if filename is None:
            # go get the one with this one's name from before
            filename = self.name + '.turbhistory'
        path = os.path.join(from_dir, filename)
        if not os.path.isfile(path):
            raise ValueError('No controller save history found at ' + path)
        with open(path) as f:
            self.state_history = json.loads(f.read())
        self.ever_updated = False


class ParamEstTurbCtrlr(TurbController):
    def __init__(self, setpoint=0.7, init_od=1e-6, init_k=None, output_limits=(0, 180), db_session=None):
        super(ParamEstTurbCtrlr, self).__init__(setpoint, init_od, output_limits=output_limits, db_session=db_session)
        self.default_k = .5
        if init_k is None:
            init_k = self.default_k
        self.k_estimate = init_k
        self.state.update({'k_estimate': init_k})
        self.k_limits = .05, 3
        self.plate = 'None'
        self.well_id = 'No ID'

    def predict_od(self, od_now, transfer_vol_frac, dt, k):
        # delta time (dt) is in seconds, k is in hr^-1
        logging.info("debug od_now")
        logging.info(od_now)
        logging.info("debug dt")
        logging.info(dt)
        logging.info("debug k")
        logging.info(k)
        logging.info("debug transfer vol frac")
        logging.info(transfer_vol_frac)
        return od_now*np.exp(dt/3600*k)/(1+transfer_vol_frac)

    def infer_k(self, od_then, transfer_vol_frac, od_now, dt):
        '''
        Predicts the OD, where od_now refers to the last known OD, whereas od_then refers to the last
        known od, before the current measurement was taken.
        '''
        min_k, max_k = self.k_limits
        logging.info("infer k calculation parameters")
        logging.info(transfer_vol_frac)
        logging.info(transfer_vol_frac)
        logging.info(od_now)
        logging.info(od_then)
        logging.info(dt)
        logging.info("end infer k parameters")
        return max(min_k, min(max_k, np.log((transfer_vol_frac + 1)*od_now/od_then)/dt*3600))

    def last_known_k(self):
        last_state = self.state_history[-1]
        return last_state.get('k_estimate', self.default_k)

    def _step(self, delta_time, od_meas, last_transfer_frac=None):
        prior_od = self.last_known_od()
        last_known_out = self.last_known_output()
        prior_k = self.last_known_k()
        last_state = self.state_history[-1]
        prior_out = last_known_out if last_transfer_frac is None else last_transfer_frac
        if od_meas is not None:
            prediction = self.predict_od(
                prior_od, prior_out, delta_time, prior_k)
            # max(prediction - .05, min(prediction + .05, od_meas)) # clamp based on prediction to rule out crazy readings
            self.od = od_meas
        # error = self.predict_od(prior_od, prior_out, delta_time, prior_k) - od_meas
        if self.ever_updated:  # only sensible to infer k after more than one point
            s = .15
            self.k_estimate = prior_k*(1-s) + self.infer_k(prior_od, prior_out,
                                                           self.od, delta_time)*s
            # try to close a fraction of the distance to the correct volume per iteration
            # use model to solve for perfect transfer volume, which may not be achievable
            s = .7
            transfer_vol_frac = (self.od*np.exp(delta_time/3600*self.k_estimate)
                                 / ((self.setpoint*s + prior_od*(1-s))) - 1)
        else:
            # play it safe
            self.k_estimate = prior_k
            transfer_vol_frac = prior_out
        # limit output
        min_out, max_out = self.output_limits
        transfer_vol_frac = min(max_out, max(min_out, transfer_vol_frac))
        self.state.update({'k_estimate': self.k_estimate})
        return transfer_vol_frac

    def set_od(self, od):
        self.od = od

    

In [ ]:
class DiagnosticPIDTurbController(TurbController):
    def __init__(self, setpoint=0.7, init_od=1e-6, kp=1.0, ki=0.1, kd=0.05, 
                 output_limits=(0, 2), db_session=None, well_id="unknown"):
        """
        Diagnostic PID-based turbidostat controller with extensive logging.
        """
        super(DiagnosticPIDTurbController, self).__init__(setpoint, init_od, output_limits=output_limits, db_session=db_session)
        
        # PID parameters
        self.kp = kp
        self.ki = ki
        self.kd = kd
        
        # PID state variables
        self.integral = 0.0
        self.previous_error = 0.0
        self.previous_time = None
        
        # Integral windup prevention
        self.integral_limits = (-5.0, 5.0)
        
        # Growth rate estimation
        # init_k = np.linspace(0.1, 3.0, 96*2)
        self.estimated_growth_rate = .5#  init_k[self.id_counter-1]  # hr^-1
        self.growth_rate_history = []
        
        # Diagnostic tracking
        self.well_id = well_id
        self.step_count = 0
        self.convergence_threshold = 0.05  # Within 5% of setpoint
        self.converged_steps = 0
        self.diagnostic_log = []
        
        # Add PID parameters to initial state
        self.state.update({
            'kp': kp, 'ki': ki, 'kd': kd,
            'integral': self.integral,
            'previous_error': self.previous_error,
            'estimated_growth_rate': self.estimated_growth_rate,
            'well_id': self.well_id
        })

    def _estimate_growth_rate(self, od_prev, od_current, dt_hours, prev_transfer_frac):
        """Estimate growth rate with diagnostic logging"""
        if dt_hours <= 0 or od_prev <= 0:
            return self.estimated_growth_rate
        
        # Account for dilution
        od_after_dilution = od_prev / (1 + prev_transfer_frac)
        
        if od_after_dilution > 0:
            try:
                k_apparent = np.log(od_current / od_after_dilution) / dt_hours
                k_apparent = max(0, min(3.0, k_apparent))
                
                self.growth_rate_history.append(k_apparent)
                if len(self.growth_rate_history) > 10:
                    self.growth_rate_history.pop(0)
                
                old_rate = self.estimated_growth_rate
                self.estimated_growth_rate = np.mean(self.growth_rate_history)
                
                # Log significant changes in growth rate
                if abs(self.estimated_growth_rate - old_rate) > 0.1:
                    print(f"Well {self.well_id}: Growth rate changed from {old_rate:.3f} to {self.estimated_growth_rate:.3f}")
                
            except (ValueError, ZeroDivisionError):
                print(f"Well {self.well_id}: Error calculating growth rate")
        
        return self.estimated_growth_rate

    def _calculate_steady_state_transfer(self, current_od, target_od, dt_hours):
        """Calculate steady-state transfer with bounds checking"""
        if dt_hours <= 0:
            return 0
        
        growth_factor = np.exp(self.estimated_growth_rate * dt_hours)
        steady_state_transfer = growth_factor - 1
        
        # Error-based correction
        error = current_od - target_od
        error_correction = error / target_od if target_od > 0 else 0
        
        corrected_transfer = steady_state_transfer + error_correction
        
        return max(0, corrected_transfer)

    def _step(self, delta_time, od_meas, last_transfer_frac=None):
        """Enhanced _step with comprehensive diagnostics"""
        self.step_count += 1
        prev_od = self.od
        
        # Update OD measurement
        if od_meas is not None:
            self.od = od_meas
        
        # Time calculations
        dt_hours = delta_time / 3600.0
        
        # Growth rate estimation
        if last_transfer_frac is not None and self.ever_updated:
            self._estimate_growth_rate(prev_od, self.od, dt_hours, last_transfer_frac)
        
        # Calculate error
        error = self.od - self.setpoint
        error_percent = (error / self.setpoint) * 100 if self.setpoint > 0 else 0
        
        # Track convergence
        if abs(error_percent) < self.convergence_threshold * 100:
            self.converged_steps += 1
        else:
            self.converged_steps = 0
            
        # PID calculations
        proportional = self.kp * error
        
        # Integral with windup protection
        if dt_hours > 0:
            self.integral += error * dt_hours
            min_int, max_int = self.integral_limits
            was_clamped = False
            if self.integral < min_int or self.integral > max_int:
                was_clamped = True
            self.integral = max(min_int, min(max_int, self.integral))
        integral_term = self.ki * self.integral
        
        # Derivative
        if self.previous_time is not None and dt_hours > 0:
            derivative = (error - self.previous_error) / dt_hours
        else:
            derivative = 0.0
        derivative_term = self.kd * derivative
        
        # PID correction
        pid_correction = proportional + integral_term + derivative_term
        
        # Feed-forward
        feedforward_transfer = self._calculate_steady_state_transfer(
            self.od, self.setpoint, dt_hours)
        
        # Final transfer calculation
        pid_scaled = pid_correction * 0.1
        transfer_vol_frac = feedforward_transfer + pid_scaled
        transfer_vol_frac = max(0, transfer_vol_frac)
        
        # Check for saturation
        min_out, max_out = self.output_limits
        was_saturated = transfer_vol_frac > max_out or transfer_vol_frac < min_out
        transfer_vol_frac_clamped = min(max_out, max(min_out, transfer_vol_frac))
        
        # Store diagnostic information
        diagnostic_info = {
            'step': self.step_count,
            'well_id': self.well_id,
            'od': self.od,
            'setpoint': self.setpoint,
            'error': error,
            'error_percent': error_percent,
            'dt_hours': dt_hours,
            'growth_rate': self.estimated_growth_rate,
            'proportional': proportional,
            'integral': self.integral,
            'integral_term': integral_term,
            'derivative_term': derivative_term,
            'pid_correction': pid_correction,
            'feedforward': feedforward_transfer,
            'transfer_calculated': transfer_vol_frac,
            'transfer_final': transfer_vol_frac_clamped,
            'was_saturated': was_saturated,
            'integral_clamped': was_clamped if 'was_clamped' in locals() else False,
            'converged_steps': self.converged_steps
        }
        
        self.diagnostic_log.append(diagnostic_info)
        
        # Print diagnostics for problematic cases
        if self.step_count % 10 == 0 or abs(error_percent) > 20:  # Every 10 steps or large error
            print(f"Well {self.well_id} Step {self.step_count}:")
            print(f"  OD: {self.od:.3f} (target: {self.setpoint:.3f}), Error: {error_percent:.1f}%")
            print(f"  Growth rate: {self.estimated_growth_rate:.3f} hr⁻¹")
            print(f"  PID components - P: {proportional:.4f}, I: {integral_term:.4f}, D: {derivative_term:.4f}")
            print(f"  Feed-forward: {feedforward_transfer:.4f}, Final transfer: {transfer_vol_frac_clamped:.4f}")
            if was_saturated:
                print(f"  WARNING: Output saturated! Calculated: {transfer_vol_frac:.4f}, Limits: {self.output_limits}")
            if was_clamped if 'was_clamped' in locals() else False:
                print(f"  WARNING: Integral clamped! Value: {self.integral:.4f}, Limits: {self.integral_limits}")
            print()
        
        # Update state
        self.previous_error = error
        self.previous_time = self.state.get('update_time', time.time())
        
        self.state.update({
            'error': error,
            'proportional': proportional,
            'integral': self.integral,
            'integral_term': integral_term,
            'derivative': derivative,
            'derivative_term': derivative_term,
            'pid_correction': pid_correction,
            'feedforward_transfer': feedforward_transfer,
            'estimated_growth_rate': self.estimated_growth_rate,
            'final_transfer': transfer_vol_frac_clamped,
            'step_count': self.step_count
        })
        
        return transfer_vol_frac_clamped

    def get_diagnostic_summary(self):
        """Return summary of controller performance"""
        if not self.diagnostic_log:
            return "No diagnostic data available"
        
        recent_errors = [abs(log['error_percent']) for log in self.diagnostic_log[-10:]]
        avg_recent_error = np.mean(recent_errors)
        
        saturated_steps = sum(1 for log in self.diagnostic_log if log['was_saturated'])
        saturation_rate = saturated_steps / len(self.diagnostic_log) * 100
        
        return {
            'well_id': self.well_id,
            'total_steps': self.step_count,
            'current_od': self.od,
            'current_error_percent': recent_errors[-1] if recent_errors else 0,
            'avg_recent_error_percent': avg_recent_error,
            'converged_steps': self.converged_steps,
            'saturation_rate_percent': saturation_rate,
            'estimated_growth_rate': self.estimated_growth_rate,
            'final_integral': self.integral
        }

    def export_diagnostic_log(self, filename=None):
        """Export diagnostic log to CSV for analysis"""
        if filename is None:
            filename = f"pid_diagnostics_well_{self.well_id}.csv"
        
        df = pd.DataFrame(self.diagnostic_log)
        df.to_csv(filename, index=False)
        print(f"Diagnostic log exported to {filename}")
        
    # Include all the other methods from the previous implementation
    def reset_pid(self):
        self.integral = 0.0
        self.previous_error = 0.0
        self.previous_time = None
        self.growth_rate_history = []
        self.step_count = 0
        self.converged_steps = 0
        self.diagnostic_log = []

    def manual_tune_conservative(self):
        self.kp = 2.0
        self.ki = 0.5
        self.kd = 0.1
        
    def set_od(self, od):
        self.od = od

In [ ]:
async def mix(lh, plate, cycles, volume, positions):
    '''
    Mixes a plate at up to 8 positions for a certain number of cycles, for a certain volume,
    assumes that there are already 8 tips on machine head.
    '''
    for _ in range(cycles):
        await lh.aspirate(plate[positions], vols=(volume))
        await lh.dispense(plate[positions], vols=(volume))




In [ ]:
async def clean_tips(lh, wash):
    '''
    Assumes that the lh, has 8 dirty tips that are currently attached to the head of the machine
    will run through the cleaning process with 2x bleach washes, then as many water washes as there are water_plates


    This method also impliments a wait to allow excess liquid to drip, as well as air blowout to ensure that no liquid 
    makes its way up into the tips
    '''
    for i in range(2):
        await lh.aspirate(wash['A2']*8, vols=[200]*8)
        await lh.dispense(wash['A2']*8, vols=[200]*8)

    for i in range(3,5):
        await lh.aspirate(wash[f'A{i}']*8, vols=[200]*8)
        await lh.dispense(wash[f'A{i}']*8, vols=[200]*8)


In [ ]:
async def dilute_96w_plate(cycle_idx, replacement_volumes, tip_rack, plate_stack: ResourceStack, plate_lid_loc: ResourceStack, media_stack: ResourceStack, media_lid_loc: ResourceStack, lh, wash, cols_of_none):
    '''
    Removes lids of the designated culture plate and media plate, dilutes the given bacterial plate (all cols minus the cols_of_none (which empty columns). after diluting each column, clean the tips to ensure no bacterial well hopping or media contamination.)
    '''
    
    # remove the lid on top of the plate to the other plate,
    # await lh.move_lid(lid=plate_stack.get_top_item(), to=plate_lid_loc)

    # remove the media lid to the top of the other media container
    # await lh.move_lid(lid=media_stack.get_top_item(), to=media_lid_loc)
    
    plate = plate_stack.get_top_item()
    media = media_stack.get_top_item()

    for col_num in range(12 - cols_of_none):
        array_idxs = [col_num*8 + i for i in range(8)]  # set the indexes for the replacement volumes & plates
        
        # want to get volumes first, to determine if picking up tips is needed for partially full plates
        volumes = [replacement_volumes[array_idxs[i]] for i in range(8)]

        for i in range(8):
            if volumes[i] is None:
                volumes[i] = 0
    
        positions = [f'{cptl_alphabet[i]}{col_num+1}' for i in range(8)]
        await lh.pick_up_tips(tip_rack[positions])

        # aspirate media from media reservoir
        await lh.aspirate(media[positions], vols=volumes)

        # dispense into the culture plate
        await lh.dispense(plate[positions], vols=volumes)

        # mix the bacterial colonies 
        await mix(cycles=2, plate=plate, positions=positions, volume=volumes, lh=lh)

        await lh.aspirate(plate[positions], vols=volumes)

        # now to dispense the spent media (with bacteria) into waste
        await lh.dispense(wash['A1']*8, vols=volumes)

        # cleaning proceedure
        await clean_tips(lh=lh, wash=wash)


        # drop clean tips back into rack
        await lh.drop_tips(tip_spots=tip_rack[positions], use_channels=[i for i in range(8)])

    # add the lids back to the culture and media plates respectivly.
    # await lh.move_lid(lid=media_lid_loc.get_top_item(), to=media_stack)
    # await lh.move_lid(lid=plate_lid_loc.get_top_item(), to=plate_stack)

    
def read_plate(plate_stack, plate_reader, is_sim):
    '''
    Reads data from the supplied plate reader and then returns the an array of values; oriented columnwise.
    '''
    # Return fake data that matches the format of the real data (ie using PlateData format)
    plate_reader.aquire_data(plate_stack=plate_stack)

    if is_sim:
        fname = 'data.csv'
        df = pd.read_csv(fname, index_col=0, skiprows=9)
    else:
        fname = plate_reader.get_output_file()

    plate_data = []
    for col in df.columns:
        plate_data.extend(df[col].tolist())
    plate_data = [float(val[5:]) if str(val).lower() != 'nan' or val is not None else 0.0 for val in plate_data]

    return plate_data
    

def calculate_concentrations_from_od(plate_data_od, slope, intercept):
    '''
    Converts all values in the plate_data_od (a 1D list) into concentration values
    based on the slope and interccept provided.
    '''
    
    for od in plate_data_od:
        concentration = od * slope + intercept
        od = concentration

    return plate_data_od


def set_standard_curve_params(known_conc, absorbances):
    '''
    Completes a linear regression to establish a concentration based on the OD. 

    returns the calculated intercept and slope.
    '''
    n = len(known_conc)
    if n != len(absorbances):
        raise IndexError('lists (absorbances, known concentrations) must have the same length')
    
    intercept, slope = np.linalg.lstsq(absorbances, known_conc, rcond=None)[0]

    return intercept, slope


def calculate_replacement_volumes(turb_controller_batch, plate_data, slope, intercept, well_volume, max_well_volume, cols_of_none):
    # Call the step function of each controller in the batch and return a list of replacement volumes

    # now plate data is a columnwise list of data for each of the wells.
    # dt of 1200 is 20 mins (1200 seconds)

    max_frac = (max_well_volume - well_volume) / well_volume
    flow_rates = [controller(od_meas=reading, delta_time=1200) for controller, reading in zip(turb_controller_batch, plate_data)]        # step (__call__()) all controllers
    
    print(f'flow rates: {flow_rates}')
    
    # ensure that dilutions don't overfill cells: max volume -> 300 µL

    replacement_volumes = []
    for frac in flow_rates:
        frac = min(max_frac, frac)
        replacement_volumes.append(frac)


    return replacement_volumes




In [ ]:
def print_channels_tip_origin(lh):
    # Prints the origin location of all tips currently on the robot
    cur_pipetter = lh.head

    for channel in cur_pipetter:
        print(f"Channel {channel}:")

        tip_tracker = lh.head[channel]
        
        if tip_tracker.has_tip == True:
            print(tip_tracker.get_tip_origin())
        else:
            print("No tip present.")
        print()

In [ ]:
async def refill_media(media_stack, lid_loc, lh):    
    if isinstance(media_stack.get_top_item(), Lid):
        await lh.move_lid(lid=media_stack.get_top_item(), to=lid_loc)
        moved_lid = True
        # now there should be no lid on this item
    if isinstance(media_stack.get_top_item(), Lid):
        raise ValueError('Too many lids on Stack to refill media')
    for j in range(1, 13):
        for i in range(8):
            media_stack.get_top_item()[f'{cptl_alphabet[i]}{j}'][0].tracker.set_liquids([("media", 2000)])
    # if moved_lid:
    #     await lh.move_lid(lid=lid_loc.get_top_item(), to=media_stack)


In [ ]:
async def empty_waste(wash):
    for j in range(1, 13):
        for i in range(8):
            # Using the f-string allows us to iterate through 
            # wells A1 -> B1 -> C1 -> ... -> H1
            # all in one line of code!
            wash['A1'][0].tracker.set_liquids([("media", 0)])

Main Robot Method to Follow:

In [ ]:
async def turb_main_loop(cycles):
    # --------------------------------
    # FLAGS/system settings
    is_simulated_plate_reader_data = True
    # TODO: make this a system argument -> flag
    std_slope = 0.252
    std_intercept = 0.089
    well_volume = 150   # µL

    # colums without any colonies (starting from left)
    # in final program will be flag set to zero unless otherwise noted 
    cols_of_none = 0

    max_well_volume = 350

    file_out_for_plate_reader = 'data.csv'

    setpoint = 0.7

    # --------------------------------

    # TODO
    # REMEMBER TO INSTANTIALIZE & CLEAR THE DB SESSION, OR IMPORT IT HERE!
    # currently done earlier in ipynb, make into method and plate here

    # --------------------------------
    data_sim = DataSimulator(session=session, rand_nor_mean=1, rand_nor_std=0.05)

    plate_reader = PlateReader(is_sim=is_simulated_plate_reader_data,
                            output_file=file_out_for_plate_reader, session=session, data_sim=data_sim)


    lh = await setup_lh()
    idx = 0
    tip_racks, wash, media_plates, culture_plates = await deck_setup(lh=lh)
    media_stacks, culture_stacks = setup_stacks(media_plates=media_plates, culture_plates=culture_plates, lh=lh)   

    # adding plates do DB
    for plate in culture_plates:
        db_add_plate(session=session, plate=plate)
        db_add_wells_from_plate(session=session, plate_name=plate.name)


    num_plates = len(culture_plates)
    num_media_plates = len(media_plates)

    max_transfer_frac = (max_well_volume - well_volume) / max_well_volume

    turb_controllers = [DiagnosticPIDTurbController(setpoint=setpoint, init_od=1e-6, output_limits=(0, max_transfer_frac), db_session=session) for i in range(96*num_plates)]
    turb_controller_batches = []
    for i in range(num_plates): 
        turb_controller_batches.append(turb_controllers[i*96:(i+1)*96])


    # continued turbidostat
    while idx < cycles:
        cycle_idx = idx % num_plates

        tip_rack = tip_racks[cycle_idx]
        plate_stack = culture_stacks[cycle_idx]
        plate_lid_loc = culture_stacks[(idx + 1) % num_plates]

        # TODO: configure testing to switch when media plate wells are below max_transfer_vol?
        media_stack = media_stacks[idx % num_media_plates]
        media_lid_loc = media_stacks[(idx + 1) % num_media_plates]

        turb_controller_batch = turb_controller_batches[cycle_idx]
        plate_data = read_plate(plate_stack=plate_stack, plate_reader=plate_reader, is_sim=is_simulated_plate_reader_data)

        replacement_fractions = calculate_replacement_volumes(turb_controller_batch, plate_data, std_slope, std_intercept, well_volume=well_volume, cols_of_none=cols_of_none, max_well_volume=max_well_volume)


        for count, (od, controller) in enumerate(zip(plate_data, turb_controller_batch)):
            fraction = max(0.0, replacement_fractions[count])

            k_est = controller.estimated_growth_rate

            row_index = count % 8        # rows A-H (in numerical form)
            col_index = (count // 8) + 1 # columns 1-12

            db_add_reading(
                session=session,
                reading=(od, k_est, fraction),
                well_row=row_index,
                well_col=col_index,
                plate_name=plate_stack.children[0].name  # if using plain plate, use plate.name = "culture_plate_0"
            )
        
        await dilute_96w_plate(cycle_idx, well_volume*replacement_fractions, lh=lh, media_lid_loc=media_lid_loc, plate_lid_loc=plate_lid_loc, cols_of_none=cols_of_none, tip_rack=tip_rack, plate_stack=plate_stack, media_stack=media_stack, wash=wash)

        # TODO: insert if statement regarding when to refill media
        await refill_media(media_stack=media_stack, lh=lh, lid_loc=media_lid_loc)
        
        # TODO: insert if statement regarding when to empty waste
        await empty_waste(wash=wash)
        # TODO: do multithreading
        idx += 1


In [ ]:
await turb_main_loop(20)

In [ ]:
'''# Graph data
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import os

# Connect to your local SQLite database
conn = sqlite3.connect("turbidostat.db")

# Query joined data from the readings, wells, and plates
query = """
SELECT
  plates.name AS plate_name,
  wells.row,
  wells.col,
  readings.timestamp,
  readings.od
FROM readings
JOIN plates ON readings.plate_id = plates.id
JOIN wells ON readings.well_id = wells.id
ORDER BY readings.timestamp
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Convert timestamps to datetime objects
df["time"] = pd.to_datetime(df["timestamp"], unit="s")
df["col"] = df["col"].astype(int)
df["well"] = df["row"] + df["col"].astype(str)

# Create output directory if it doesn't exist
os.makedirs('od_sim_plots', exist_ok=True)
directory = 'od_sim_plots'
existing_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
plot_count = len(existing_files)

# Define rows and columns for plate layout
rows = "ABCDEFGH"
cols = range(1, 13)

df.to_csv('sim_data_df_from_query.csv')

# Group data by plate name and plot each separately
for plate_index, (plate_name, plate_df) in enumerate(df.groupby("plate_name")):
    fig, axes = plt.subplots(8, 12, figsize=(24, 16), sharex=True, sharey=True)
    fig.suptitle(f"OD Over Time per Well — Plate: {plate_name}", fontsize=18)

    for i, row in enumerate(rows):
        for j, col in enumerate(cols):
            ax = axes[i, j]
            well_id = f"{row}{col}"
            well_df = plate_df[(plate_df["row"] == row) & (plate_df["col"] == col)]

            if not well_df.empty:
                ax.plot(well_df["time"], well_df["od"], linestyle='-', linewidth=1)
                ax.set_ylim(0,1.0)

            ax.set_title(well_id, fontsize=8)
            ax.tick_params(labelsize=6)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    filename = f"{directory}/sim_attempt_{plot_count + plate_index + 1}_{plate_name}.png"
    plt.savefig(filename)
    plt.show()
'''

In [ ]:
'''import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os

# Seaborn style
sns.set(style="whitegrid", context="notebook")

# Connect to your local SQLite database
conn = sqlite3.connect("turbidostat.db")

# Query joined data from the readings, wells, and plates
query = """
SELECT
  plates.name AS plate_name,
  wells.row,
  wells.col,
  readings.timestamp,
  readings.od
FROM readings
JOIN plates ON readings.plate_id = plates.id
JOIN wells ON readings.well_id = wells.id
ORDER BY readings.timestamp
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Convert timestamps to datetime objects
df["time"] = pd.to_datetime(df["timestamp"], unit="s")
df["col"] = df["col"].astype(int)
df["well"] = df["row"] + df["col"].astype(str)

# Create output directory
os.makedirs('od_sim_plots', exist_ok=True)
directory = 'od_sim_plots'
existing_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
plot_count = len(existing_files)

# Define rows and columns for plate layout
rows = "ABCDEFGH"
cols = range(1, 13)

df.to_csv('sim_data_df_from_query.csv', index=False)

# Group data by plate name and plot each separately
for plate_index, (plate_name, plate_df) in enumerate(df.groupby("plate_name")):
    fig, axes = plt.subplots(8, 12, figsize=(24, 16), sharex=True, sharey=True)
    fig.suptitle(f"OD Over Time per Well — Plate: {plate_name}", fontsize=18)

    for i, row in enumerate(rows):
        for j, col in enumerate(cols):
            ax = axes[i, j]
            well_df = plate_df[(plate_df["row"] == row) & (plate_df["col"] == col)]

            if not well_df.empty:
                sns.lineplot(data=well_df, x="time", y="od", ax=ax, linewidth=1)
                ax.set_ylim(0, 1.0)

            ax.set_title(f"{row}{col}", fontsize=8)
            ax.tick_params(labelsize=6)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    filename = f"{directory}/sim_attempt_{plot_count + plate_index + 1}_{plate_name}.png"
    plt.savefig(filename)
    plt.show()
'''